依赖项 | 版本--- | ---
SatPy | 0.9.0PyResample | 1.10.1Trollimage | 1.5.3PyKdtree | 1.3.1

# 快速入门 EARS-NWC

<p>2012年12月，<a class="reference external" href="http://www.eumetsat.int/">欧洲气象卫星组织（EUMETSAT）</a>扩展了<a class="reference external" href="http://www.eumetsat.int/home/main/satellites/groundnetwork/earssystem/index.htm">欧洲气象卫星地面接收系统（EARS）</a>服务，新增了一项服务，旨在为欧洲用户提供云信息支持。来自近实时的极轨卫星。</p><p>此 <em>EARS-NWC</em> 服务提供参数 <em>云掩码</em>、<em>云类型</em> 和<em>云顶温度和高度 (CTTH)</em> 通过使用极地平台得出的系统 (<a class="reference external" href="http://nwcsaf.smhi.se/">PPS</a>) 软件包来自 <a class="reference external" href="http://www.nwcsaf.org/">NWC SAF</a>。这些产品是通过...推导得出的。来自EARS核心站接收的AVHRR数据，其接收覆盖范围包括欧洲和北大西洋。产品通过<a class="reference external" href="http://www.eumetsat.int/home/main/dataaccess/eumetcast/index.htm">EUMETCast</a>（EUMETSAT <a class="reference external" href="http://www.eumetsat.int/home/main/dataaccess/eumetcast/receptionstationset-up/sp_20100623124251305?l=en">数据通道1</a>）分发，时效性优于30分钟，并以netCDF4格式提供。地理定位信息以控制点网格形式提供，并存储在每个产品中。<p>当前为该服务提供支持的卫星是Metop-B和NOAA-19。</p>

## 加载和连接云类型数据粒

In [1]:
from satpy import Scene
from satpy import find_files_and_readers
from datetime import datetime

DATA_DIR = "/home/a000680/laptop/Nordisk/EARS_NWC/data/case_20180314/ears"

myfiles = find_files_and_readers(base_dir=DATA_DIR,
                                 start_time=datetime(2018, 3, 14, 10, 5),
                                 end_time=datetime(2018, 3, 14, 10, 11),
                                 reader='nc_nwcsaf_pps')
scene = Scene(filenames=myfiles)
scene.load(['ct'])

(0, ['/home/a000680/usr/src/satpy/satpy/etc/readers/nc_nwcsaf_pps.yaml'], ['nc_nwcsaf_pps'])


/usr/lib64/python2.7/_abcoll.py:391: FutureWarning: iteration over an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Iterate over the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  return list(self)
/usr/lib64/python2.7/_abcoll.py:391: FutureWarning: calling len() on an xarray.Dataset will change in xarray v0.11 to only include data variables, not coordinates. Call len() on the Dataset.variables property instead, like ``len(ds.variables)``, to preserve existing behavior in a forwards compatible manner.
  return list(self)
/home/a000680/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


现在我们已经加载并拼接了由上方开始和结束时间指定的时间窗口内的Cloudtype产品数据粒度中的<em>ct</em>字段

In [2]:
print(scene['ct'].data.shape)
print(scene['ct'].data.compute())

(4320, 2048)
[[ 2  2  2 ...  6  6  6]
 [ 2  2  2 ...  6  6  6]
 [ 2  2  2 ... 10  6  6]
 ...
 [ 7  6  6 ...  2  2  2]
 [ 7  6  6 ...  2  2  2]
 [ 7  6  6 ...  2  2  2]]


<p>此外，地理定位功能也已解除限制。也就是说，这是完整的分辨率地理位置信息已被从控制点网格中重新生成在控制点上对经度和纬度进行插值和外推网格。这是通过使用 <a class="reference external" href="http://www.github.com/adybbroe/python-geotiepoints">python-geotiepoints</a> 工具实现的，但这是对用户透明:</p>

In [3]:
print(scene['ct'].area.lats.shape)

(4320, 2048)


现在，让我们使用从文件中读取的Nowcasting SAF调色板来可视化云类型数据：

In [4]:
scene.load(['cloudtype'])
scene.show('cloudtype')

<img src='https://github.com/pytroll/pytroll-examples/raw/master/satpy/ears_nwc_cloudtype_20180314T1005Z_20180314T1010Z_thumb.png'>

# 加载所有CTTH数据粒，拼接，重投影，生成合成图像，并保存到磁盘

In [5]:
from glob import glob
import os.path

In [6]:
DATA_DIR = "/home/a000680/laptop/Nordisk/EARS_NWC/data/case_20180314/ears"

## 列出目录中所有的CTTH文件，包括bz2压缩和解压后的文件：

In [7]:
myfiles = glob(os.path.join(DATA_DIR, "*CTTH_C*.nc.bz2"))
myfiles = myfiles + glob(os.path.join(DATA_DIR, "*CTTH_C*.nc"))

## 生成场景对象

In [8]:
scene = Scene(filenames=myfiles, reader='nc_nwcsaf_pps')

## 加载云高合成图像

In [9]:
scene.load(['cloud_top_height'])

/home/a000680/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in less_equal
  return func(*args2)
/home/a000680/.local/lib/python2.7/site-packages/dask/local.py:271: RuntimeWarning: invalid value encountered in greater_equal
  return func(*args2)


## 将数据重新采样到预定义的 'euron1' 区域

In [10]:
lscn = scene.resample('euron1', radius_of_influence=5000)

No handlers could be found for logger "pyresample.kd_tree"


In [11]:
lscn.show('cloud_top_height')

<img src='https://github.com/pytroll/pytroll-examples/raw/master/satpy/ears_nwc_ctth_20180314T1000Z_20180314T1014Z_thumbnail.png'>

## 将图像保存到文件

In [11]:
dt_start = lscn.attrs['start_time'].strftime('%Y%m%dT%H%MZ')
dt_end = lscn.attrs['end_time'].strftime('%Y%m%dT%H%MZ')
lscn.save_dataset(
    'cloud_top_height', filename='./ears_nwc_ctth_{starttime}_{endtime}.png'.format(starttime=dt_start, endtime=dt_end))